# グリッドサーチ

ハイパーパラメータ最適化手法

## グリッドサーチ

In [1]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
import numpy as np

# データセットの読み込み
data = load_breast_cancer()

# 特徴量
X = data.data          # NumPy配列（shape: [n_samples, n_features]）
# ラベル（0: 悪性, 1: 良性）
y = data.target # LabelEncoder使う必要なし

from sklearn.model_selection import train_test_split
# データセットを訓練用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

pipe_svc = make_pipeline(
    StandardScaler(),
    SVC(random_state=1)
)
param_range = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = [{
    'svc__C': param_range,
    'svc__kernel': ['linear']
}, {
    'svc__C': param_range,
    'svc__gamma': param_range,
    'svc__kernel': ['rbf']
}]

gs = GridSearchCV(estimator=pipe_svc,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=10,
                  refit=True, # 最適なパラメータで再学習
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)
print('Best score: %.3f' % gs.best_score_)
print('Best parameters: %s' % gs.best_params_)

Best score: 0.976
Best parameters: {'svc__C': 10, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}


In [3]:
# refit=Trueにしていれば必要ない
clf = gs.best_estimator_
clf.fit(X_train, y_train)
print('Test accuracy: %.3f' % clf.score(X_test, y_test))

Test accuracy: 0.982


## ランダムサーチ

グリッドサーチはしらみつぶし探索であるためコストが高い。  
ランダムサーチは分布（分布は指定できる）からランダムに抽出する。コスト的にも時間的にもグリッドサーチより効率的。

In [4]:
import scipy
param_range = scipy.stats.loguniform(1e-4, 1e3)
np.random.seed(1)
param_range.rvs(10)

array([8.30145146e-02, 1.10222804e+01, 1.00184520e-04, 1.30715777e-02,
       1.06485687e-03, 4.42965766e-04, 2.01289666e-03, 2.62376594e-02,
       5.98924832e-02, 5.91176467e-01])

In [5]:
from sklearn.model_selection import RandomizedSearchCV
pipe_svc = make_pipeline(
    StandardScaler(),
    SVC(random_state=1)
)
param_grid = [{
    'svc__C': param_range,
    'svc__kernel': ['linear']
}, {
    'svc__C': param_range,
    'svc__gamma': param_range,
    'svc__kernel': ['rbf']
}]
rs = RandomizedSearchCV(estimator=pipe_svc,
                        param_distributions=param_grid,
                        scoring='accuracy',
                        cv=10,
                        refit=True, # 最適なパラメータで再学習
                        n_iter=20, # ランダムサーチの試行回数
                        n_jobs=-1)
rs = rs.fit(X_train, y_train)
print('Best score: %.3f' % rs.best_score_)
print('Best parameters: %s' % rs.best_params_)

Best score: 0.969
Best parameters: {'svc__C': np.float64(4.934834261073333), 'svc__kernel': 'linear'}


## Successive Having

* ランダムサンプリングを使って設定候補を大量に抽出する。
* 制限されたリソースで（例えば、訓練データセット全体ではなくそのごく一部を使って）モデルを訓練する。
* 予測性能の低いものから順に50%の設定候補を捨てる。
* 手順2に戻って、リソースの量を増やした上で同じ処理を繰り返す。

In [6]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
hs = HalvingRandomSearchCV(pipe_svc,
                         param_distributions=param_grid,
                         n_candidates='exhaust', # 最後のイテレーションでリソースの最大数（この場合は訓練データの個数）が使われる。
                         resource='n_samples',
                         factor=1.5,
                         random_state=1,
                         n_jobs=-1,
)
hs = hs.fit(X_train, y_train)
print('Best score: %.3f' % hs.best_score_)
print('Best parameters: %s' % hs.best_params_)

# HalvingGridSearchCVの結果を使って再学習
clf = hs.best_estimator_
print('Test accuracy: %.3f' % clf.score(X_test, y_test))

Best score: 0.974
Best parameters: {'svc__C': np.float64(0.05971247755848463), 'svc__kernel': 'linear'}
Test accuracy: 0.982


ハイパーパラメータ最適化のためのライブラリとしては、hyperoptがよく知られている。  
TPE(Tree-structured Parzen Estimators：確率的モデルに基づくベイズ最適化手法)を含め、何種類か実装されている。

**ベイズ最適化（Bayesian Optimization）**

* **概要**: ガウス過程などで目的関数をモデル化し、次に評価すべき点を予測。
* **利点**: 少ない試行回数で高性能な結果を得られることが多い。
* **欠点**: 実装がやや複雑で、計算コストもやや高い。
* **ツール例**: `Optuna`, `Hyperopt`, `scikit-optimize`


**進化的アルゴリズム（Evolutionary Algorithms）**

* **概要**: 遺伝的アルゴリズムや遺伝的プログラミングなど。
* **利点**: 離散・連続混在パラメータや多峰性問題に強い。
* **ツール例**: `DEAP`, `Nevergrad`


**ハイパーバンド / サクセッショナルハルチューニング（Hyperband, Successive Halving）**

* **概要**: 計算資源を動的に割り当て、早期停止を活用して非効率なモデルを排除。
* **利点**: 多くの試行を高速に実施可能。
* **ツール例**: `Ray Tune`, `Optuna`, `Auto-sklearn`

**AutoMLツールの活用**

* **概要**: モデル選定＋ハイパーパラメータ調整を自動化。
* **代表例**:

  * `Auto-sklearn`
  * `TPOT`
  * `Google AutoML`
  * `H2O.ai`


## 5x2交差検証

In [7]:
from sklearn.model_selection import cross_val_score
param_range = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = [{
    'svc__C': param_range,
    'svc__kernel': ['linear']
}, {
    'svc__C': param_range,
    'svc__gamma': param_range,
    'svc__kernel': ['rbf']
}]
gs = GridSearchCV(estimator=pipe_svc,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=2)
scores = cross_val_score(gs, X_train, y_train, scoring='accuracy', cv=5)
print(f'CV accuracy: {scores.mean():.3f} +/- {scores.std():.3f}')

CV accuracy: 0.971 +/- 0.011


In [ ]:
# 使わない場合
from sklearn.tree import DecisionTreeClassifier
gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=0),
                  param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, None]},
                  scoring='accuracy',
                  cv=2)
scores = cross_val_score(gs, X_train, y_train, scoring='accuracy', cv=5)
print(f'CV accuracy: {scores.mean():.3f} +/- {scores.std():.3f}')

# 前者の方が性能が良い

CV accuracy: 0.932 +/- 0.013
